In [8]:
"""# Importing Libraries"""
import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt
import scipy
from scipy.interpolate import interp1d
from scipy.stats import kurtosis
from scipy.stats import skew
from sklearn import preprocessing
from sklearn.preprocessing import normalize
from scipy.ndimage import zoom
from skimage.measure import block_reduce
from skimage.transform import resize
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.optimizers import Adam
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Flatten, Dense, Activation
from tensorflow.keras.utils import to_categorical
from keras.layers.core import Reshape
from keras.layers import LSTM

tf.compat.v1.enable_eager_execution()

In [2]:
"""# Loading Data"""

path = "../../Cog_DataSets/testing/"
# path = "drive/MyDrive/Colab Notebooks/CogAge/Datasets/testing/"
test_acc = np.load(path+"testAccelerometer.npy")
test_grav = np.load(path+"testGravity.npy")
test_gyro = np.load(path+"testGyroscope.npy")
test_jinsAcc = np.load(path+"testJinsAccelerometer.npy")
test_jinsGyro = np.load(path+"testJinsGyroscope.npy")
test_Label =np.load(path+"testLabels.npy") 
test_linAcc = np.load(path+"testLinearAcceleration.npy")
test_MsAcc = np.load(path+"testMSAccelerometer.npy")
test_MsGyro = np.load(path + "testMSGyroscope.npy")
test_MsMag = np.load(path+"testMagnetometer.npy")
# test_acc

path = "../../Cog_DataSets/training/"
# path = "drive/MyDrive/Colab Notebooks/CogAge/Datasets/training/"
train_acc = np.load(path+"trainAccelerometer.npy")
train_grav = np.load(path+"trainGravity.npy")
train_gyro = np.load(path+"trainGyroscope.npy")
train_jinsAcc = np.load(path+"trainJinsAccelerometer.npy")
train_jinsGyro = np.load(path+"trainJinsGyroscope.npy")
train_Label =np.load(path+"trainLabels.npy") 
train_linAcc = np.load(path+"trainLinearAcceleration.npy")
train_MsAcc = np.load(path+"trainMSAccelerometer.npy")
train_MsGyro = np.load(path + "trainMSGyroscope.npy")
train_MsMag = np.load(path+"trainMagnetometer.npy")
# print(train_Label.shape)

In [3]:
"""# Reshape and stack Data Before Fitting to Model"""
# changing shape of sensor data to (#,400,3)
# downsample
train_acc_reshaped = block_reduce(train_acc, block_size=(1, 2, 1), func=np.mean)
train_gyro_reshaped = block_reduce(train_gyro, block_size=(1, 2, 1), func=np.mean)
train_grav_reshaped = block_reduce(train_grav, block_size=(1, 2, 1), func=np.mean)
train_linAcc_reshaped = block_reduce(train_linAcc, block_size=(1, 2, 1), func=np.mean)
# upsample
train_MsAcc_reshaped = resize(train_MsAcc, (2284, 400, 3), mode='edge')
train_MsGyro_reshaped = resize(train_MsGyro, (2284, 400, 3), mode='edge')
# upsample
train_MsMag_reshaped = np.repeat(train_MsMag, 2, axis=1)
train_jinsAcc_reshaped = np.repeat(train_jinsAcc, 5, axis=1)
train_jinsGyro_reshaped = np.repeat(train_jinsGyro, 5, axis=1)

test_acc_reshaped = block_reduce(test_acc, block_size=(1, 2, 1), func=np.mean)
test_gyro_reshaped = block_reduce(test_gyro, block_size=(1, 2, 1), func=np.mean)
test_grav_reshaped = block_reduce(test_grav, block_size=(1, 2, 1), func=np.mean)
test_linAcc_reshaped = block_reduce(test_linAcc, block_size=(1, 2, 1), func=np.mean)
test_MsAcc_reshaped = resize(test_MsAcc, (2288, 400, 3), mode='edge')
test_MsGyro_reshaped = resize(test_MsGyro, (2288, 400, 3), mode='edge')
test_MsMag_reshaped = np.repeat(test_MsMag, 2, axis=1)
test_jinsAcc_reshaped = np.repeat(test_jinsAcc, 5, axis=1)
test_jinsGyro_reshaped = np.repeat(test_jinsGyro, 5, axis=1)

# all data of shape #,400,3
# adding all relative data.
# Mobile training accelerometer + Mobile testing accelerometer data
train_acc_reshaped = np.append(train_acc_reshaped,test_acc_reshaped, axis=0)
train_gyro_reshaped = np.append(train_gyro_reshaped,test_gyro_reshaped, axis=0)
train_grav_reshaped = np.append(train_grav_reshaped,test_grav_reshaped, axis=0)
train_linAcc_reshaped = np.append(train_linAcc_reshaped,test_linAcc_reshaped, axis=0)
train_MsAcc_reshaped = np.append(train_MsAcc_reshaped,test_MsAcc_reshaped, axis=0)
train_MsGyro_reshaped = np.append(train_MsGyro_reshaped,test_MsGyro_reshaped, axis=0)
train_MsMag_reshaped = np.append(train_MsMag_reshaped,test_MsMag_reshaped, axis=0)
train_jinsAcc_reshaped = np.append(train_jinsAcc_reshaped,test_jinsAcc_reshaped, axis=0)
train_jinsGyro_reshaped = np.append(train_jinsGyro_reshaped,test_jinsGyro_reshaped, axis=0)


print("Shape of all sensors after up/down sample... ", train_acc_reshaped.shape, train_gyro_reshaped.shape, train_grav_reshaped.shape, train_linAcc_reshaped.shape
                       , train_MsAcc_reshaped.shape, train_MsGyro_reshaped.shape, train_MsMag_reshaped.shape,
                       train_jinsAcc_reshaped.shape, train_jinsGyro_reshaped.shape)

# all data of shape 4572,400,3
# 4572 = 2284(training) + 2288(testing)


# stack
all_data = np.stack([train_acc_reshaped, train_gyro_reshaped, train_grav_reshaped, train_linAcc_reshaped
                       , train_MsAcc_reshaped, train_MsGyro_reshaped, train_MsMag_reshaped,
                       train_jinsAcc_reshaped, train_jinsGyro_reshaped], axis=-1)

all_Label = np.append(train_Label, test_Label, axis=-1)


Shape of all sensors after up/down sample...  (4572, 400, 3) (4572, 400, 3) (4572, 400, 3) (4572, 400, 3) (4572, 400, 3) (4572, 400, 3) (4572, 400, 3) (4572, 400, 3) (4572, 400, 3)


In [5]:
# 70% training data + labels
train_data = all_data[: int(all_data.shape[0]*0.7)]
# 30% testing data + labels
test_data = all_data[int(all_data.shape[0]*0.7):]
train_labels = all_Label[: int(all_Label.shape[0]*0.7)]
test_labels = all_Label[int(all_Label.shape[0]*0.7):]

print("\nShape of training and testin data + labels...\n")
print(train_data.shape, test_data.shape)
print(train_labels.shape, test_labels.shape)


# Split the data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(train_data, train_labels, test_size=0.2, random_state=42)

print("Shape of training + validation Data")
print(np.shape(x_train), np.shape(y_train))

y_train = to_categorical(y_train, num_classes=55)
y_val = to_categorical(y_val, num_classes=55)

print("Shape of training + validation Labels")
print(np.shape(y_train), np.shape(y_val))



Shape of training and testin data + labels...

(3200, 400, 3, 9) (1372, 400, 3, 9)
(3200,) (1372,)
Shape of training + validation Data
(2560, 400, 3, 9) (2560,)
Shape of training + validation Labels
(2560, 55) (640, 55)


In [18]:
# HYPER PARAMTERS

# Filter parameters, i.e. about the number of inputs processed by each neuron of the convolutional layer
filter1Size = (11,1)
filter2Size = (13,1)
filter3Size = (13,1)

# Downsampling factors of the pooling layers
poolingLayer1Factor = (2,1)
poolingLayer2Factor = (3,1)
poolingLayer3Factor = (2,1)

# Number of feature maps processed by each convolutional layer
nbFeaturesLayer1 = 50
nbFeaturesLayer2 = 40
nbFeaturesLayer3 = 30

# Activation function of the convolutional layer(s)
activationConv = 'relu'

# Output dimension of the LSTM
outputLSTM = 100

# Parameters of the dense layer
activationMLP = 'relu'
inputMLP = 500

# Training parameters
batchSize = 400
numberOfEpochs = 30
learningRate = 0.001

input_shape = (400,3,9)
nbClasses = 55
timeWindow = 400
nbSensors = 9

#-------------------------------------------------------------------------------------------------------
# normConv1Lstm: define a batch normalization + 2 convolutional/max-pooling + LSTM DNN
#-------------------------------------------------------------------------------------------------------
def normConv1Lstm(
    inputShape,
    nkerns,
    filterSizes,
    poolSizes,
    activationConv,
    timeWindow,
    nbSensors,
    outputLSTM,
    inputMLP,
    activationMLP,
    nbClasses,
    withSoftmax=True):

    outputSizeLastConv = int((timeWindow-filterSizes[0][0]+1)/poolSizes[0][0])
    #outputSizeLastConv = (timeWindow-filterSizes[0][0]+1)

    model = Sequential()

    # Batch normalization layer
    model.add(BatchNormalization(input_shape=inputShape))

    # Convolution + max-pooling layers
    model.add(Conv2D(nkerns[0], kernel_size=filterSizes[0], activation=activationConv))
    model.add(MaxPooling2D(pool_size=poolSizes[0]))

    # model.add(Conv2D(nkerns[0], kernel_size=filterSizes[0], activation='linear', input_shape=inputShape))
    # model.add(BatchNormalization())
    # model.add(Activation(activationConv))
    # model.add(MaxPooling2D(pool_size=poolSizes[0]))
    print(model.layers[-1].output_shape)
    # LSTM layer with a many-to-one implementation
    # Note: size of the output = (outputSizeLastConv, outputLSTM)
    model.add(Reshape((outputSizeLastConv,3*nkerns[0]))) 
    model.add(LSTM(outputLSTM,return_sequences=False))
    print('LSTM output size: ---------------------------')
    print(model.layers[-1].output_shape)

    # Fully-connected layer
    model.add(Dense(inputMLP, activation=activationMLP))
    print('Dense layer size: ---------------------------')
    print(model.layers[-1].output_shape)
    #model.add(Dropout(0.5))

    # Softmax layer
    if withSoftmax:
        model.add(Dense(nbClasses, activation='softmax'))

    # Print the summary of the model
    model.summary

    # Return the model
    return model

model = normConv1Lstm(
    inputShape=(400,3,9),
    nkerns=[nbFeaturesLayer1, nbFeaturesLayer2, nbFeaturesLayer3],
    filterSizes=[filter1Size, filter2Size, filter3Size],
    poolSizes=[poolingLayer1Factor, poolingLayer2Factor, poolingLayer3Factor],
    activationConv='relu',
    timeWindow=400,
    nbSensors=9,
    outputLSTM=outputLSTM,
    inputMLP=inputMLP,
    activationMLP=activationMLP,
    nbClasses=55,
    withSoftmax=True)

(None, 195, 3, 50)
LSTM output size: ---------------------------
(None, 100)
Dense layer size: ---------------------------
(None, 500)


2023-05-09 17:08:51.206410: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-09 17:08:51.210573: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-09 17:08:51.213741: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [19]:
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(learning_rate=0.001),
    metrics=['accuracy']
)

In [20]:
history = model.fit(
    x_train, y_train, 
    validation_data=(x_val, y_val),
    epochs=numberOfEpochs,
    batch_size=batchSize
)

Epoch 1/30


2023-05-09 17:09:39.071769: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 110592000 exceeds 10% of free system memory.
2023-05-09 17:09:39.666727: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-09 17:09:39.670801: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-09 17:09:39.673760: I tensorflow/core/common_runtime/executor.cc:1197] [/dev

7/7 [==============================] - ETA: 0s - loss: 3.9366 - accuracy: 0.0492

2023-05-09 17:09:51.581736: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-09 17:09:51.583943: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-09 17:09:51.585710: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

7/7 [==============================] - 13s 1s/step - loss: 3.9366 - accuracy: 0.0492 - val_loss: 3.8028 - val_accuracy: 0.0750
Epoch 2/30
7/7 [==============================] - 9s 1s/step - loss: 3.6467 - accuracy: 0.1113 - val_loss: 3.5064 - val_accuracy: 0.1250
Epoch 3/30
7/7 [==============================] - 9s 1s/step - loss: 3.2738 - accuracy: 0.1984 - val_loss: 3.1716 - val_accuracy: 0.1797
Epoch 4/30
7/7 [==============================] - 10s 1s/step - loss: 2.9080 - accuracy: 0.2414 - val_loss: 2.9871 - val_accuracy: 0.2078
Epoch 5/30
7/7 [==============================] - 10s 1s/step - loss: 2.6241 - accuracy: 0.3164 - val_loss: 2.8104 - val_accuracy: 0.2625
Epoch 6/30
7/7 [==============================] - 10s 1s/step - loss: 2.5087 - accuracy: 0.3309 - val_loss: 2.8394 - val_accuracy: 0.2281
Epoch 7/30
7/7 [==============================] - 10s 1s/step - loss: 2.4931 - accuracy: 0.3215 - val_loss: 2.8469 - val_accuracy: 0.2438
Epoch 8/30
7/7 [==============================]